In [ ]:
import folium

In [ ]:
from io import StringIO
import numpy as np
import pandas as pd
latitude_longitude_csv_str =\
"""\
city,latitude,longitude
London,51.5072,-0.1275
Birmingham,52.48,-1.9025
Manchester,53.4794,-2.2453
Leeds,53.7997,-1.5492
Newcastle,55.0077,-1.6578
Birstall,52.6736,-1.12
Glasgow,55.8609,-4.2514
Liverpool,53.4,-2.9833
Portsmouth,50.8058,-1.0872
Southampton,50.9,-1.4
Nottingham,52.95,-1.15
Bristol,51.45,-2.5833
Sheffield,53.3833,-1.4667
Kingston upon Hull,53.7444,-0.3325
Leicester,52.6333,-1.1333
Edinburgh,55.953,-3.189
Caerdydd,51.4833,-3.1833
Stoke-on-Trent,53,-2.1833
Coventry,52.4081,-1.5106
Reading,51.4542,-0.9731\
"""
latitude_longitude_pd = pd.read_csv(StringIO(latitude_longitude_csv_str), index_col='city')
latitude_longitude_deg = latitude_longitude_pd[['latitude', 'longitude']].to_numpy()

In [ ]:
def ll_to_distance_matrix(latitude_longitude_deg):
    la_lo_rad = np.pi * latitude_longitude_deg / 180.0
    la, lo = la_lo_rad[:, 0], la_lo_rad[:, 1]
    earth_radius = 6370.0
    distance_matrix = earth_radius * np.arccos(np.sin(la[:, np.newaxis]) * np.sin(la[np.newaxis, :]) + np.cos(la[:, np.newaxis]) * np.cos(la[np.newaxis, :]) * np.cos(lo[:, np.newaxis] - lo[np.newaxis, :]))
    return distance_matrix

In [ ]:
from networkx import path_weight, from_numpy_matrix
from networkx.algorithms.tree.mst import minimum_spanning_tree
from networkx.algorithms.traversal.depth_first_search import dfs_preorder_nodes, dfs_labeled_edges, dfs_edges

def get_cities(latitude_longitude_deg, city_list):

    # Calculate distance matrix 
    distance = ll_to_distance_matrix(latitude_longitude_deg)

    # Create weighted undirected graph G
    G = from_numpy_matrix(distance)

    # MST using Kruskal Algorithm
    M = minimum_spanning_tree(G, algorithm='kruskal')
    
    # DFS order of cities
    dfs = list(dfs_preorder_nodes(M)) + [0]

    # List of vertices (cities)
    ordered_city_list = [city_list[vertex] for vertex in dfs]

    return ordered_city_list

In [ ]:
# Map
m = folium.Map(location=[53, -1], zoom_start=6)

# Marker Cluster
for city, lat, long in latitude_longitude_pd.itertuples():
    folium.Marker(location=[lat, long], tooltip=f'{city}').add_to(m)

# Line
folium.PolyLine(locations=latitude_longitude_pd.loc[ordered_city_list + ['London']].values).add_to(m)

display(m)

- Click [here](https://nbviewer.org/github/rawat9/side-projects/blob/main/20-UK-cities/20_UK_cities.ipynb) to view the map